## Part 1

Firstly we do the monte carlo simulation to find the mean at least tiles to make percolation(assume the result is $k$) 

In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

def dfs(grid, visited, x, y, n) -> bool: 
    DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]   
    stack = [(x, y)]
    while stack:
        cx, cy = stack.pop()
        if visited[cx, cy]:
            continue
        visited[cx, cy] = True
        for dx, dy in DIRECTIONS:
            nx, ny = cx + dx, cy + dy
            if (0 <= nx < n and 0 <= ny < n) and not visited[nx, ny] and grid[nx, ny] == 1:
                if nx == n - 1:  
                    return True
                stack.append((nx, ny))
    return False

def percolates(grid, n) -> bool:
    visited = np.zeros((n, n), dtype=bool)
    for col in range(n):
        if grid[0, col] == 1 and not visited[0, col] and dfs(grid, visited, 0, col, n):
            return True
    return False

def monte_carlo_simulation(n) -> Tuple[float, int]:
    grid = np.zeros((n, n), dtype=int)  
    open_sites = 0
    while True:
        x, y = random.randint(0, n - 1), random.randint(0, n - 1)
        while grid[x, y] == 1:  
            x, y = random.randint(0, n - 1), random.randint(0, n - 1)
        grid[x, y] = 1
        open_sites += 1
        if percolates(grid, n):
            return open_sites / (n * n), open_sites  

def estimate_percolation_threshold(n, trials)-> Tuple[float, int]:
    thresholds = []
    nums = []
    for _ in range(trials):
        threshold, num = monte_carlo_simulation(n)
        thresholds.append(threshold)
        nums.append(num)
    return np.mean(thresholds), int(np.mean(nums))



In [9]:
n = 20  
trials = 10 


estimated_threshold, mean_nums_tile = estimate_percolation_threshold(n, trials)
print(f"mean threshold:{estimated_threshold}\nmean nums:{mean_nums_tile}")

mean threshold:0.6094999999999999
mean nums:243


From above we know that if $n=20$, $k=237$

In [10]:
n = 50  
trials = 10 


estimated_threshold, mean_nums_tile = estimate_percolation_threshold(n, trials)
print(f"mean threshold:{estimated_threshold}\nmean nums:{mean_nums_tile}")

mean threshold:0.58908
mean nums:1472


From above we know that if $n=50$, $k=1480$

In [11]:
n = 100 
trials = 10 


estimated_threshold, mean_nums_tile = estimate_percolation_threshold(n, trials)
print(f"mean threshold:{estimated_threshold}\nmean nums:{mean_nums_tile}")

mean threshold:0.59401
mean nums:5940


From above we know that if $n=100$, $k=5903$



Then suppose there are at least $k$ tiles opened to make a percolation. And the probability of each tile opened is $p$. So the probability of percolation is $\sum_{i=1}^{n^2-k+1} = (\frac{p}{1-p})^ip^{k-1}(1-p)^{n^2-k+1} = \frac{p^k(1-p)^{n^2-k}(1-(\frac{p}{1-p})^{n^2-k+1})}{1-\frac{p}{1-p}}$


# Part 2

## Task 1


$=\dfrac{P(\sigma=+1,\boldsymbol{\sigma}_ {-k}  )}{P(\boldsymbol{\sigma}_{-k} )}$

$=\dfrac{P(\sigma=+1,\boldsymbol{\sigma}_ {-k}  )}{P(\boldsymbol{\sigma}_{-k},\sigma_k=+1)+P(\boldsymbol{\sigma}_{-k},\sigma_k=-1)}$

<br>

Let $\boldsymbol{\sigma_1}=(\boldsymbol{\sigma_{-k} },\sigma_k=+1),\boldsymbol{\sigma_2}=(\boldsymbol{\sigma_{-k} },\sigma_k=-1)$

$H(\boldsymbol{\sigma_1 })=\sum_{(v,w)\in \Omega}{\sigma_v \sigma_w}$

$=H(\boldsymbol{\sigma_k} )-\sum_{v\sim k}{\sigma_k \sigma_v}$

$=H(\boldsymbol{\sigma_k} )-\sum_{v\sim k}{\sigma_v}$

Similarly,$H(\boldsymbol{\sigma_2 })=H(\boldsymbol{\sigma_k} )+\sum_{v\sim k}{\sigma_v}$

$\Rightarrow H(\boldsymbol{\sigma_2 })=H(\boldsymbol{\sigma_1 })+2\sum_{v\sim k}{\sigma_v}$

<br>

$\therefore P(\sigma_k=+1|\sigma_{-k})$

$=\dfrac{e^{-\beta H(\boldsymbol{\sigma_1}) } }{e^{-\beta H(\boldsymbol{\sigma_1}) } +e^{-\beta H(\boldsymbol{\sigma_2}) } }$

$=\dfrac{e^{-\beta H(\boldsymbol{\sigma_1}) } }{e^{-\beta H(\boldsymbol{\sigma_1}) } +e^{-\beta H(\boldsymbol{\sigma_1 })+2\sum_{v\sim k}{\sigma_v} } }$

$=\dfrac{1}{1+e^{-2\beta\sum_{v\sim k}{\sigma_v} } }$

<br>

$\therefore P(\sigma_k=-1|\sigma_{-k})=1-P(\sigma_k=+1|\sigma_{-k})$

=$1-\dfrac{1}{1+e^{-2\beta\sum_{v\sim k}{\sigma_v} } }$

=$\dfrac{e^{-2\beta\sum_{v\sim k}{\sigma_v} }}{1+e^{-2\beta\sum_{v\sim k}{\sigma_v} } }$

$=\dfrac{1}{1+e^{2\beta\sum_{v\sim k}{\sigma_v} }}$